In [ ]:
import re
import xml.etree.ElementTree as ET
import stanza
import pandas as pd
import numpy as np
import nltk
from nltk import ngrams, Text, bigrams, BigramCollocationFinder 
from nltk.collocations import BigramAssocMeasures
import collections
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
import itertools
import matplotlib.pyplot as plt
import lxml
from collections import Counter
from sklearn import preprocessing
from gensim import utils
from sklearn.decomposition import PCA
from time import time  
import gensim.models
import gensim.downloader as api
import matplotlib.pyplot as plt
import numpy as np
import json
from scipy.spatial.distance import cosine
from sklearn import cluster
import plotly.express as px
import string

In [ ]:
#read_file
list_of_file = {}
from os import listdir
for file in listdir("/home/scc/sergio.zanotto/bin/Identities/Impaqts/discorsi_off/"):
    if file != ".ipynb_checkpoints":
        with open("/home/scc/sergio.zanotto/bin/Identities/Impaqts/discorsi_off/" + file, 'r', encoding='utf-8', errors = "ignore") as f:
                lines = f.readlines()
                list_of_file[file] = lines

df = pd.DataFrame(columns=['Filename', 'Text'])
for file, lines in list_of_file.items():
    text_list = []
    fil = re.sub('.txt', '', file)
    text_list.append([line.strip() for line in lines])  # Each line is a separate list within the cell
    df = df.append({'Filename': fil, 'Text': text_list}, ignore_index=True)

In [ ]:
## calculating author when missing
gg = []
for x in df["Filename"]:
    g = False
    for y,z in zip(dd["NOME"],dd["PARLANTE"]):
        if x==y:
            gg.append(z)
            g = True
    if g == False: 
        gg.append("None")
df["author"] = gg     

In [ ]:
## cleaning the dataset from possible interruptions
corpus_speech = []
interruption = []
for filename, text, author in zip(df["Filename"], df["Text"], df["author"]):
    first_interrup = 0
    interruptions = 0
    text_list = []
    current_author = True
    mention = False
    #print(filename,author)
    if author != "None":
        author = author.upper()
        author1 = re.sub(r'[^A-Za-z]', '', author)
        for y in text:
            for x in y:   
                x = re.sub("\/", '', x)
                x = re.sub("\n", '', x)
                x = re.sub(r"\+", '', x)
                x = re.sub("\ufeff", '', x)
                words = x.strip().split()
                if words:
                    first_word = re.sub(r'[^A-Za-z]', '', words[0])
                    if (
                        (words[0].isupper() and words[0].endswith((".", ":"))) or
                        (len(words) > 1 and words[1].isupper() and words[1].endswith((".", ":"))) or
                        (len(words) > 2 and words[2].isupper() and words[2].endswith((".", ":")))
                    ):
                        if words[0].isupper():
                            if first_word in author1:
                                mention = True
                                current_author = True
                                author_words = author.split()
                                for auth in author_words:
                                    x = x.replace(auth, '')  
                            else:
                                current_author = False 
                                interruptions += 1
                    else:
                        if current_author == False:
                            if mention == False:
                                if (
                                    (words[0].isupper() and words[0].endswith((".", ":"))) or
                                    (len(words) > 1 and words[1].isupper() and words[1].endswith((".", ":"))) or
                                    (len(words) > 2 and words[2].isupper() and words[2].endswith((".", ":")))
                                ):
                                    current_author = False
                                else:
                                    current_author = True                            
                    if current_author == True:
                        text_list.append(x)
    #print(text_list ,len(text_list))
    interruption.append(interruptions)                        
    corpus_speech.append(text_list)

df["Text"] = corpus_speech
df["interruption"] = interruption

## Calculating Features

In [ ]:
# Creating a zip file for the UD_PROFILING tool
df = df[df['Text'].apply(lambda x: len(x) > 0)]

df['Text'] = df['Text'].apply(lambda x: '\n'.join(x))

# Writing each row's text to a text file named after the 'Filename' column
for _, row in df.iterrows():
    with open(f"/home/scc/sergio.zanotto/bin/Identities/Impaqts/Identities/txt/{row['Filename']}.txt", "w") as file:
        file.write(row['Text'])
# The files will be saved with names from the 'Filename' column
import zipfile

# Path for the zip file
zip_path = 'text_files.zip'


with zipfile.ZipFile(zip_path, 'w') as zipf:
    for _, row in df.iterrows():
        # Adding each text file to the zip
        text_file_path = f"/home/scc/sergio.zanotto/bin/Identities/Impaqts/Identities/txt/{row['Filename']}.txt"
        zipf.write(text_file_path, arcname=row['Filename']+'.txt')

In [ ]:
# Convert dictionary to DataFrame for applying measures

dd = pd.read_csv("Monitoring.csv", encoding="latin-1") #not sharable
popu = pd.read_csv('populism-anticentrism.csv', encoding='latin-1' #Decadri_Boussalis, 2020
AoA = pd.read_csv('AoA_data.csv', encoding='latin-1') # Montefinese et Al., 2019
Concrete_it = pd.read_csv('concreteness_it.csv', encoding='latin-1') #Gregori et Al., 2020



In [ ]:
## ficing concreteness
desired_columns = ["TARGET","CONC_AVG"]

# Filter the DataFrame to keep only the desired columns
Concrete_it = Concrete_it[desired_columns].copy()
tt = Concrete_it["TARGET"].values.tolist()
concrete_set = set(tt)
conrete_value = {}
for z in tt:
    value = 0
    n = 0
    for x,y in zip(Concrete_it["TARGET"],Concrete_it["CONC_AVG"]):
        if z == x:
            n += 1
            value += y
    conrete_value[z]= value/n

In [ ]:
## AoA
AoA_words = AoA["words"].values.tolist()

In [ ]:
## Populist words
from numpy import nan

# Remove nan values from the list
populist_words = [word for word in populist_words if not pd.isnull(word)]


In [ ]:
### calculate G_Index, Type_Token_Ration, Populist_Words
G_index_list = []
Type_token_ratio_list = []
pop_words_ratio_list = []

# Initialize lists to store calculated metrics
G_index_list = []
Type_token_ratio_list = []
pop_words_ratio_list = []

# Loop through each row in the DataFrame
for x, y in zip(df["Text"], df["interruption"]):
    # Initialize counters and storage for types
    types = []
    n_sent = len(x) - y
    n_word = 0
    n_character = 0
    n_pop_word = 0
    n_word1 = 0
    
    # Process each sentence in the text
    for m in x:
        # Clean the text and split into words
        m = re.sub('[^\w\s]', ' ', m)
        m = m.split()
        
        # Update word counts and type list
        n_word += len(m)
        for el in m:
            n_character += len(el)
            types.append(el.lower())
            
            # Check for populist words and update counts
            if el not in italian_stop_words:
                n_word1 += 1
                for pop_word in populist_words:
                    if pop_word in el:
                        n_pop_word += 1
    
    # Calculate G_index using the formula
    G_index = 89 + ((300 * n_sent - 10 * n_character) / n_word)
    G_index_list.append(G_index)
    
    # Calculate Type-Token Ratio
    Type_token_ratio = len(set(types)) / n_word
    Type_token_ratio_list.append(Type_token_ratio)
    
    # Calculate Populist Words Ratio
    pop_words_ratio = n_pop_word / n_word1
    pop_words_ratio_list.append(pop_words_ratio)

# Assign calculated metrics to DataFrame columns
df["G_index"] = G_index_list
df["Type_token_ratio"] = Type_token_ratio_list
df["Populist_words_ratio"] = pop_words_ratio_list

In [ ]:
nlp = stanza.Pipeline(lang='it') ## using Stanza for PoS

In [ ]:
#function for different measures, but we used only Concreteness
pos = ["VERB","AUX"]
def measures(x1):
    x = ' '.join(x1)
    x2 = nlp(x)
    n_pass = 0
    n_verbs = 0
    n_sub = 0
    new_text = []
    concr_lev = 0
    n_word_pres = 0
    n_word = 0
    non_pres_word = 0
    for sent in x2.sentences: 
        for word in sent.words:
            if word.text.lower() not in italian_stop_words:  
                n_word += 1
                new_text.append(word.lemma)
                word_is_present = False
                for key, value in conrete_value.items():
                    if word.lemma.lower() == key:
                        word_is_present = True
                        n_word_pres += 1
                        concr_lev += value    
                if word_is_present == False:
                    non_pres_word += 1
            if word.upos in pos:
                n_verbs += 1
                if ":pass" in word.deprel:
                    n_pass += 1
                if word.feats:
                    if "Mood=Sub" in word.feats or "Mood=Cnd" in word.feats:
                        n_sub += 1                    
    ratepassvsact = n_pass / n_verbs if n_verbs != 0 else 0  
    ratesub = n_sub / n_verbs if n_verbs != 0 else 0
    clev = concr_lev/n_word_pres if n_word_pres != 0 else 0
    perc_not_pres_word = (non_pres_word/n_word)*100 if n_word != 0 else 0
    return ratepassvsact, ratesub, new_text, clev, perc_not_pres_word 

In [ ]:
## AoA and COncreteness
ratepassvsact_list= [] #not_used
ratesub_list= [] #not_used
new_text_list= []
dict_emb_list= [] #not_used
AoA_list = []
concr_list = []
i = 0
for x in df["Text"]:
    i += 1
    n_word_inlistlemmas = 0
    non_present_word = 0
    present_word = 0
    AoA_score = 0
    ratepassvsact, ratesub, new_text, concr_lev, percentage_non_present_words_in_concrete = measures(x)
    
    #append elements
    ratepassvsact_list.append(ratepassvsact*100)
    ratesub_list.append(ratesub*100)
    concr_list.append(concr_lev)
    
    ##AoA
    new_text = [''.join(char for char in s if char not in string.punctuation) for s in new_text]
    new_text = [s for s in new_text if s]
    for word in new_text:     
        n_word_inlistlemmas += 1
        word_is_present = False
        for key, value in zip(AoA["words"],AoA["values"]):
            if word == key:
                word_is_present = True
                AoA_score += value
        if word_is_present == False:
            non_present_word += 1
        else:
            present_word += 1
    AoA_mean = AoA_score/present_word if present_word != 0 else 0
    percentage_non_present_words = (non_present_word/n_word_inlistlemmas)*100 if n_word_inlistlemmas != 0 else 0
    
    #append AOA
    AoA_list.append(AoA_mean)     
df["AoA"] = AoA_list
df["Concreteness"] = concr_list

## Adding Monitoring of IMPAQTS (METADATA)

In [ ]:
mon = pd.read_csv("Monitoring.csv", encoding="latin-1") #Cominetti et Al, 2022

In [ ]:
#append metadata
data_en = []
sottocorp= []
sit_pol= []
channel= []
public= []
orient = []
for x in df["Filename"]:
    i = 0
    for y,a,b,c,d,e,f in zip(mon["NOME"],mon["DATA ENUNCIAZIONE"], mon["SOTTOCORPUS TEMPORALE"],mon["FORZA POLITICA"],mon["DECADE"],mon["TIPO"],mon["ORIENTAMENTO"]):
        if x == y:
            i += 1
            data_en.append(a) 
            sottocorp.append(b)
            sit_pol.append(c)
            channel.append(d)
            public.append(e)
            orient.append(f)

In [ ]:
df["date"] = data_en
df["subcorpus"] = sottocorp
df["Political_Orientation"] = orient
df["forza_politica"] = sit_pol
df["decade"] = channel
df["type"] = public

In [ ]:
# Calculating type in a readable way
new_tipo = []
for x in df["Filename"]:
    x = str(x)
    x = x[-2:]
    if "A" in x:
        x = "Assemlea"
    if "C" in x:
        x = "Comizio"
    if "P" in x:
        x = "Riunione_Partito"
    if "T" in x:
        x = "Dichiarazione_Trasmessa"
    if "F" in x:
        x = "Dichiarazione_faccia_a_faccia"
    if "N" in x:
        x = "Dichiarazione_nuovi_media"
    if "O" in x:
        x = "Riunione_operativa"
    if "I" in x:
        x = "Intervista"
    
    new_tipo.append(x)
df["type"] = new_tipo

In [ ]:
#classify populist parties (given classification in PopuList 3.0 (2023))
so_called_populists = ["FdI", "LN", "M5S", "IdV", "La Rete", "RC", "FI", "PdL"]
popll = []
for x in df["forza_politica"]:
    m = 0
    if x in so_called_populists:
        m = 1
    else:
        m = 0
    popll.append(m)

In [ ]:
df["is_populist"] = popll

In [ ]:
## Word Frequency
docs = [nlp(text) for text in df["Text"] ]
filtered_tokens_per_doc = [[token.text.lower() for token in doc if token.pos_ in ["NOUN", "VERB"]] for doc in docs]
# Define the path to your text file
file_path = 'it_50k.txt'

freq = pd.read_csv(file_path, sep=" ", header=None, names=['Word', 'Frequency'])
freq['Rank'] = freq['Frequency'].rank(method='dense', ascending=False).astype(int) #ranking words for frequency
freq['Decile'] = pd.qcut(freq['Rank'], q=10, labels=False, duplicates='drop') + 1 #assigning decile to ranks (10 + out-of-vocabulary 11)
freq["weight"] = [x / 10 for x in freq['Decile']] #assigning a weight to smooth decile importance (especially the 11th)

## calculate
freq_dict = dict(zip(freq["Word"], zip(freq["Decile"], freq["weight"])))

df_mean_score = []
non_present_words_list = []
adjusted_non_present_score_list = []

for doc in filtered_tokens_per_doc:
    total_score, non_present_words = 0, 0
    
    for word in doc:
        if word in freq_dict:
            decile, weight = freq_dict[word]
            score = decile * weight
        else:
            non_present_words += 1
    
    # Calculate the percentage of non-present words
    n_tokens = len(doc) if doc else 1  # Prevent division by zero if doc is empty
    non_present_percentage = (non_present_words / n_tokens * 100)
    
    # Adjust the score for non-present words based on their percentage
    adjusted_non_present_score = 11 * (0.1 * (non_present_percentage / 100)) #non present words get 11th percentile but the lowest weight, multiplied for how many they are (the more unseen words, the more they count)
    
    # Recalculate the total score with adjusted score for non-present words
    for word in doc:
        if word in freq_dict:
            decile, weight = freq_dict[word]
            score = decile * weight
        else:
            score = adjusted_non_present_score  # Use the adjusted score for non-present words
        total_score += score
    
    mean_score = total_score / n_tokens
    adjusted_non_present_score_list.append(adjusted_non_present_score)
    
    df_mean_score.append(mean_score)
    non_present_words_list.append(non_present_percentage)
df["word_frequency"] = df_mean_score #the higer the score, the LESS frequent a word is

In [ ]:
## calculate optimal Topic x Decade (fixing decade)
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords

# nltk.download('stopwords')
italian_stop_words = stopwords.words('italian')

filtered_tokens_per_doc = [[token.text.lower() for token in doc if token.pos_ in ["NOUN"]] for doc in docs]
filtered_tokens_per_doc = [' '.join(token) for token in filtered_tokens_per_doc]

# Adjusting the decade column
df['decade'] = df['decade'].replace({'2020-2022': '2020-2023'})

# Ensure correct order of decades
decade_order = ['1990-1999', '2000-2009', '2010-2019', '2020-2023']

# Order for political orientation
orientation_order = ['S', 'CS', 'I', 'C', 'CD', 'D']


# Vectorization
tfidf_vectorizer = TfidfVectorizer(stop_words=italian_stop_words, max_features=10000)
tfidf_matrix = tfidf_vectorizer.fit_transform(df["Text"])

# Finding the optimal number of clusters using the elbow method
wcss = []  # Within-cluster sum of squares
for i in range(1, 50):  # Let's try from 1 to 10 clusters
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=42)
    kmeans.fit(tfidf_matrix)
    wcss.append(kmeans.inertia_)

# Plotting the results
plt.plot(range(1, 50), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')  # Within cluster sum of squares
plt.show()

In [ ]:
# The result is a sparse matrix. To convert it to a dense format and put it in a DataFrame:
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# If you wish to include the document identifiers or any specific indexing, adjust accordingly:
tfidf_df.index = df.index

# Number of top words to retrieve
top_n = 10  

# Initialize an empty list to store the lists of important words for all documents
important_words_all_docs = []

# Iterate through each row/document in tfidf_df to find the top N important words
for index, row in tfidf_df.iterrows():
    # Sort the row to get the indices (words) of the top N values (TF-IDF scores)
    top_words_indices = row.sort_values(ascending=False).head(top_n).index
    # Convert indices to a list and append to the main list
    important_words_all_docs.append(list(top_words_indices))

#print(important_words_all_docs)
df["tfidfwords"] = important_words_all_docs
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np

# Adjusting the decade column
df['decade'] = df['decade'].replace({'2020-2022': '2020-2023'})

# Ensure correct order of decades
decade_order = ['1990-1999', '2000-2009', '2010-2019', '2020-2023']

# Order for political orientation
orientation_order = ['S', 'CS', 'I', 'C', 'CD', 'D']

# No filtering step based on party document counts

# Create a unique color for each party
unique_parties = df['forza_politica'].unique()
party_colors_hex = {party: plt.cm.tab20(i / len(unique_parties)) for i, party in enumerate(unique_parties)}

# Convert RGBA to HEX
party_colors_hex = {party: '#%02x%02x%02x' % (int(rgba[0]*255), int(rgba[1]*255), int(rgba[2]*255)) for party, rgba in party_colors_hex.items()}

def color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    party = kwargs['party']
    return party_colors_hex.get(party, 'black')  # Default to black if party not found

# Plotting
fig, axes = plt.subplots(nrows=len(orientation_order), ncols=len(decade_order), figsize=(20, 15), subplot_kw={'xticks': [], 'yticks': []})

# Ensure axes is a 2D array
if len(orientation_order) == 1:
    axes = np.expand_dims(axes, axis=0)

for (party, decade, orientation), words in df.groupby(['forza_politica', 'decade', 'Political_Orientation'])['tfidfwords'].sum().items():
    row = orientation_order.index(orientation)
    col = decade_order.index(decade)
    ax = axes[row, col] if axes.ndim > 1 else axes[col]  # Support for 1D array of axes
    
    # Create a singular bag-of-words string
    text = ' '.join(words)
    
    # Generate a word cloud
    wordcloud = WordCloud(background_color='white', width=400, height=300,
                          color_func=lambda *args, **kwargs: color_func(*args, **kwargs, party=party)).generate(text)
    
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.set_title(f"{party}\n{decade} - {orientation}")
    ax.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
from gensim.models import LdaMulticore, CoherenceModel
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Assuming 'df' is your DataFrame with 'decade' and 'text' columns
# nltk.download('stopwords')
italian_stop_words = stopwords.words('italian')

# Preprocess the text: tokenize and remove stopwords
def preprocess(text):
    return [word for word in simple_preprocess(text, deacc=True) if word not in italian_stop_words]

# Apply preprocessing
df['processed'] = filtered_tokens_per_doc
df['processed'] = df['processed'].map(preprocess)

# Determine the optimal number of topics for each decade
decades = df['decade'].unique()
coherence_values = {}
for decade in decades:
    texts = df[df['decade'] == decade]['processed'].tolist()
    dictionary = Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    coherence_scores = []
    for num_topics in range(2, 11):  # Let's try from 2 to 10 topics
        lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=42)
        coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()
        coherence_scores.append(coherence_lda)
    
    coherence_values[decade] = coherence_scores

# Plotting the coherence scores
plt.figure(figsize=(15, 7))
for decade, scores in coherence_values.items():
    plt.plot(range(2, 11), scores, marker='o', label=f'{decade}')
plt.title('LDA Topic Coherence')
plt.xlabel('Number of Topics')
plt.ylabel('Coherence Score')
plt.legend()
plt.show()


In [ ]:
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

# Since documents are already preprocessed and tokenized
def preprocess_document(tokens):
    return [word for word in tokens if word not in italian_stop_words and word.isalpha()]


# Group documents by decade and preprocess
grouped_docs = df.groupby('decade')['processed'].apply(lambda docs: [preprocess_document(tokens) for tokens in docs]).to_dict()


# Perform LDA for each decade
lda_models = {}
for decade, docs in grouped_docs.items():
    # Create a dictionary and corpus
    dictionary = Dictionary(docs)
    corpus = [dictionary.doc2bow(doc) for doc in docs]
    
    # Number of topics for this decade
    num_topics = optimal_topics[decade]
    
    # LDA Model
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=42, passes=10)
    lda_models[decade] = lda_model
    
    # Print the topics
    print(f"\nTopics for {decade}:")
    for idx, topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=10):
        print(f"Topic {idx}: {topic}")


In [ ]:
# Initialize an empty list to hold the dominant topic for each document
dominant_topics = []

# Iterate over each document in the DataFrame
for index, row in df.iterrows():
    decade = row['decade']
    processed_doc = row['processed']
    
    # Get the appropriate LDA model and dictionary for the document's decade
    lda_model = lda_models[decade]
    dictionary = lda_model.id2word
    
    # Convert the processed document to BOW format
    bow = dictionary.doc2bow(processed_doc)
    
    # Get the topic distribution for the document
    doc_distribution = lda_model.get_document_topics(bow)
    
    # Sort the topics by probability to find the dominant one
    doc_distribution.sort(key=lambda x: x[1], reverse=True)
    
    # Take the most probable topic for the document
    dominant_topic, prob = doc_distribution[0] if doc_distribution else (None, 0)
    
    # Generate a unique name for the dominant topic incorporating the decade
    unique_topic_name = f"{decade}_Topic{dominant_topic}" if dominant_topic is not None else "No_Dominant_Topic"
    
    # Append the unique topic name to our list
    dominant_topics.append(unique_topic_name)

# Assign the dominant topic names to a new column in the DataFrame
df['Topic'] = dominant_topics

In [ ]:
#calculating parties in majority or opposition in parliament
cal = {"gennaio":1, "febbraio":2,"marzo":3,"aprile":4,"maggio":5,"giugno":6,"luglio":7,"agosto":8,"settembre":9,"ottobre":10,"novembre":11,"dicembre":12,}
mag_opp = pd.read_csv('maggioranzainparlamento.csv', encoding='latin-1')
inzio = []
fin = []
interval = []
i = 0
for x, y in zip(mag_opp["Inizio"],mag_opp["Fine"]):
    i+= 1
    x = x.split()
    if i == 68:
        y = "26 giugno 2023"
    y = y.split()
    for m in cal:
        if m == x[1]:
            g = cal[m]
        if m == y[1]:
            f = cal[m]
    xx = str(x[0]) + "/" + str(g) + "/" + str(x[2]) 
    yy = str(y[0]) + "/" + str(f) + "/" + str(y[2])
    inzio.append(xx)
    fin.append(yy)
    interval.append(pd.date_range(start=xx, end=yy))

mag_opp["Inizio"] = inzio #range
mag_opp["Fine"] = fin #range

majority = []
for i, (x, pp, decade, filename) in enumerate(zip(df["date"], df["forza_politica"], df["decade"], df["Filename"])):
    # Initialize maop as 0 at the start of each iteration
    maop = 0

    if pd.isnull(x):
        # Derive 'x' from 'filename' by removing the last 3 characters and taking the last 2 digits
        try:
            x = int(filename[:-3][-2:])
        except ValueError:
            # Log error and continue to next iteration after appending maop (which is 0)
            print(f"Invalid filename info at index {i}: filename={filename}, derived value attempted: {filename[:-3][-2:]}")
            majority.append(maop)  # Ensure a value is appended for this row
            continue
    else:
        # Extract year from 'x' when it is a date string
        try:
            x = int(pd.to_datetime(x).year)
        except ValueError:
            # Log error and continue to next iteration after appending maop (which is 0)
            print(f"Invalid date format at index {i}: date={x}, decade={decade}, forza_politica={pp}")
            majority.append(maop)  # Ensure a value is appended for this row
            continue

    # Proceed with existing logic
    for t, mag in zip(interval, mag_opp["Partito"]):
        years_range = t.year.unique()
        if x in years_range:
            parties = mag.split('-')
            if pp.strip() in [p.strip() for p in parties]:
                maop = 1
                break  # Found a matching condition, no need to check further

    # Append the maop value after processing the current row
    majority.append(maop)

# Check if the lengths match as expected
#print(f"Length of 'majority': {len(majority)}, Expected: {len(df)}")
df["is_majority"] = majority

In [ ]:
df.to_csv("offical_text_profiling_impaqts.csv", header=True, index=False, encoding="UTF-8")